In [11]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import date
import matplotlib.pyplot as plt

### get data

In [13]:
%%time 
## 20CR apcp
filelist=[]
for i in range(1981,2014,1):
    filelist.append(f'/Users/dongxiay/Documents/data/daily/apcp.{i}.nc')
ds1=xr.open_mfdataset(filelist,combine='by_coords')
ds=ds1['apcp'].loc[:,-45:0,100:160]

##Oliver MJO
df=pd.read_csv("/Users/dongxiay/Documents/data/MJO_ot12.csv")
df['time']=pd.to_datetime(df[['year','month','day']])
df=df.set_index('time').drop(columns=['year','month','day'])
df.loc[df['amplitude'].lt(1) ,'phase'] = 0

##NINO3.4
dateparser = lambda d: pd.datetime.strptime(d,'%Y%m%d')
df2=pd.read_csv('http://climexp.knmi.nl/data/inino34_daily.dat',
                skiprows=12,delim_whitespace=True,
                names=['time','enso'],
                parse_dates=['time'],
                date_parser=dateparser)
df2=df2.set_index(df2['time']).drop(columns='time')
df2.loc[df2['enso'].lt(-1) ,'watch'] = -1
df2.loc[df2['enso'].gt(1) ,'watch'] = 1
df2.loc[(df2['enso']>(-1)) & (df2['enso']<(1)),'watch']=0

##merge
dfmjo=df.loc[(df.index >= ds.time.min().values) & (df.index <= ds.time.max().values)][['amplitude','phase']]
dfenso=df2.loc[(df2.index >= ds.time.min().values) & (df2.index <= ds.time.max().values)]['watch']
data=xr.merge([ds,dfmjo.to_xarray(),dfenso.to_xarray()],join='inner').load()

data

<timed exec>:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


CPU times: user 15.7 s, sys: 1.44 s, total: 17.1 s
Wall time: 18.7 s


<xarray.Dataset>
Dimensions:    (time: 11810, lat: 46, lon: 61)
Coordinates:
  * time       (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2013-12-31
  * lat        (lat) float32 -45.0 -44.0 -43.0 -42.0 ... -3.0 -2.0 -1.0 0.0
  * lon        (lon) float32 100.0 101.0 102.0 103.0 ... 157.0 158.0 159.0 160.0
Data variables:
    apcp       (time, lat, lon) float32 0.3625 0.3125 0.2625 ... 1.025 0.875
    amplitude  (time) float64 1.773 1.754 1.641 1.486 ... 0.1155 0.2596 0.4505
    phase      (time) int64 2 2 2 2 2 2 2 2 2 2 3 3 ... 0 0 0 0 0 0 0 0 0 0 0 0
    watch      (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes: (12/14)
    long_name:         daily mean 3-hourly accumulated total precipitation at...
    units:             kg/m^2
    GRIB_name:         APCP
    var_desc:          Precipitation amount
    dataset:           NOAA/CIRES/DOE 20th Century Reanalysis version 3mo Dai...
    level_desc:        Surface
    ...                ...
    standard_name:     precipitation_amount
    valid_range:       [  0. 100.]
    statistic_method:  Ensemble mean is calculated by averaging over all 80 e...
    GridType:          Cylindrical Equidistant Projection Grid
    datum:             wgs84
    actual_range:      [ 0.    35.625]

###  Ratio=Prob/baseline for Elnino
Prob= (number of days when rainfall exceeding the 67% threshold (THR67) for MJO phase 1) / (number of days that is in MJO phase 1)
baseline= (number of days when rainfall exceeding the THR67) / (total days for all ElNino days)

In [3]:
datanino = data['apcp'].loc[data.watch== 1]
phasenino= data['phase'].loc[data.watch== 1]

In [4]:
def func(ds):
    return ds.count('time') / len(ds['time'])

In [5]:
THR=0.67
THR_threshold = datanino.where(datanino!=0).chunk(dict(time=-1)).quantile(THR, dim='time')

baseline = datanino.where(datanino >= THR_threshold).count('time') / len(datanino['time'])
baseline=baseline.compute()

result = datanino.where(datanino >= THR_threshold).groupby(phasenino).apply(func).compute()
ratio= result/baseline
obs=ratio


In [12]:
import regionmask
mask = regionmask.defined_regions.natural_earth.countries_110.mask(ratio)
AU_index = regionmask.defined_regions.natural_earth.countries_110.map_keys('AU')
ratio_AU = ratio.where(mask == AU_index)

cbar_kwargs = {'orientation':'vertical', 'shrink':0.6, 'label':'Ratio'}

ratio_AU.plot(col='phase', col_wrap=3, size=3,
             cmap='RdBu', vmin=0, vmax=2,
             cbar_kwargs=cbar_kwargs)

plt.savefig("test.pdf", format="pdf",bbox_inches = 'tight')

In [8]:
# import NCL colormap
! pip install cmaps
import cmaps

In [27]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
import cartopy.mpl.ticker as cticker 


leftlon, rightlon, lowerlat, upperlat = (110,160,-23.5,-10)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()

title=[]
for i in range(1,9,1):
    title.append(f'Phase {i}')
   
fig = plt.figure(figsize=(12,8))

for i in range(4):
    for j in range(2):
        ax = fig.add_axes([0.1+j*0.475, 1.5-i*0.28, 0.4, 0.3],projection = ccrs.PlateCarree())
#         ax.set_title('({})'.format(title[i,j]),loc='left',fontsize=14)
        ax.set_extent(img_extent, crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE) 
        ax.set_xticks(np.arange(leftlon,rightlon+10,10), crs=ccrs.PlateCarree())
        ax.set_yticks(np.arange(lowerlat,upperlat+5,5), crs=ccrs.PlateCarree())
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)
        ax.title.set_text(title[i*2+j])
        c = ax.contourf(THR_threshold['lon'],THR_threshold['lat'],ratio_AU[i*2+j+1,:,:],
                        levels=np.arange(0,2,0.1),
                        extend='both' ,
                        cmap=cmaps.precip_diff_12lev,
                        transform=ccrs.PlateCarree()) 
        
#colorbar, You would want to make sure the colorbar always sits in the same axes. 
# To this end, you may create a colorbar axes (cax) outside the loop
fig.colorbar(c, cax=fig.add_axes([0.26, 0.63, 0.5, 0.03 ]),
             orientation='horizontal',  
             label= 'Ratio', 
             ticks=np.arange(0.1,2.1,0.1),
             shrink=0.7)

### Monte Carlo 2-side test

In [46]:
phasenino=phasenino.to_dataframe()

AttributeError: 'DataFrame' object has no attribute 'to_dataframe'

In [48]:
%%time
def create_mjo_ensemble(df_, col, steps):
    df = df_.copy()
    for stp in steps:
        df[f"{col}_{stp}"] = df[col].shift(stp)
        df=df.reset_index()
        df.loc[0:stp-1, f"{col}_{stp}"] = df.loc[len(df)-stp:, f"{col}"].values
        df = df.set_index('time')
    return df
    
# add lagging 
dfmjos = create_mjo_ensemble(phasenino, 'phase', np.arange(50,len(phasenino['phase'])-50,7))

CPU times: user 556 ms, sys: 176 ms, total: 732 ms
Wall time: 860 ms


In [50]:
datam=xr.merge([datanino,dfmjos.to_xarray()],join='inner').load()
datam

<xarray.Dataset>
Dimensions:     (time: 1349, lat: 46, lon: 61)
Coordinates:
  * time        (time) datetime64[ns] 1982-05-22 1982-05-23 ... 2012-11-18
  * lat         (lat) float32 -45.0 -44.0 -43.0 -42.0 ... -3.0 -2.0 -1.0 0.0
  * lon         (lon) float32 100.0 101.0 102.0 103.0 ... 158.0 159.0 160.0
Data variables: (12/181)
    apcp        (time, lat, lon) float32 0.3375 0.325 0.3 ... 0.5 0.5 0.5625
    phase       (time) float64 1.0 1.0 2.0 4.0 4.0 4.0 ... 0.0 0.0 0.0 0.0 0.0
    phase_50    (time) float64 0.0 0.0 0.0 0.0 0.0 8.0 ... 8.0 8.0 1.0 0.0 0.0
    phase_57    (time) float64 8.0 8.0 8.0 8.0 1.0 0.0 ... 0.0 0.0 0.0 7.0 8.0
    phase_64    (time) float64 0.0 0.0 0.0 0.0 0.0 7.0 ... 8.0 8.0 8.0 1.0 1.0
    phase_71    (time) float64 8.0 8.0 8.0 8.0 8.0 1.0 ... 7.0 7.0 7.0 8.0 8.0
    ...          ...
    phase_1261  (time) float64 7.0 7.0 8.0 8.0 1.0 1.0 ... 0.0 0.0 6.0 6.0 7.0
    phase_1268  (time) float64 0.0 0.0 0.0 0.0 6.0 6.0 ... 3.0 3.0 0.0 0.0 0.0
    phase_1275  (time) float64 3.0 3.0 3.0 3.0 0.0 0.0 ... 2.0 2.0 2.0 2.0 3.0
    phase_1282  (time) float64 2.0 2.0 2.0 2.0 2.0 2.0 ... 1.0 1.0 1.0 2.0 2.0
    phase_1289  (time) float64 0.0 0.0 1.0 1.0 1.0 2.0 ... 0.0 0.0 0.0 0.0 0.0
    phase_1296  (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 8.0 8.0 8.0
Attributes: (12/14)
    long_name:         daily mean 3-hourly accumulated total precipitation at...
    units:             kg/m^2
    GRIB_name:         APCP
    var_desc:          Precipitation amount
    dataset:           NOAA/CIRES/DOE 20th Century Reanalysis version 3mo Dai...
    level_desc:        Surface
    ...                ...
    standard_name:     precipitation_amount
    valid_range:       [  0. 100.]
    statistic_method:  Ensemble mean is calculated by averaging over all 80 e...
    GridType:          Cylindrical Equidistant Projection Grid
    datum:             wgs84
    actual_range:      [ 0.    35.625]

In [55]:
PW="phase"
a=[i for i in list(datam.keys()) if i[:len(PW)]==PW]
ratiom=ratio.expand_dims({'iteration':len(a)}).copy()

In [68]:
%%time
for i, value in enumerate(a):
    THR=0.67

    THR_threshold = datam['apcp'].where(datam['apcp']!=0).chunk(dict(time=-1)).quantile(THR, dim='time')

    baseline = datam.where(datam['apcp'] >= THR_threshold)['apcp'].count('time') / len(datam['time'])
    baseline=baseline.compute()

    result = datam['apcp'].where(datam['apcp'] >= THR_threshold).groupby(datam[value]).apply(func).compute()
    ratio= result/baseline
    ratiom[i,:,:,:]=ratio
    
filename = "/Users/dongxiay/Documents/data/"+'ratiom_elnino.nc'
ratiom.to_netcdf(filename, 'w')



CPU times: user 2min 17s, sys: 2.97 s, total: 2min 20s
Wall time: 2min 19s


In [71]:
#Monte resamples
fmonte=xr.open_dataset("/Users/dongxiay/Documents/data/ratiom_elnino.nc")

In [87]:
#ranking based on the iteration dimension for each phase/lat/lon
a=fmonte['apcp'].sortby(fmonte['iteration'])

#calculate the p-value, if obs is outside of p1/p2, then mask=1 (or true)
p1=a.quantile(0.1,dim="iteration")
p2=a.quantile(0.9,dim="iteration")

sig= xr.where((obs >= p2) | (obs <= p1), 1, -1)

In [88]:
sig_AU = sig.where(mask == AU_index)
sig_AU

<xarray.DataArray 'apcp' (phase: 9, lat: 46, lon: 61)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * lat      (lat) float32 -45.0 -44.0 -43.0 -42.0 -41.0 ... -3.0 -2.0 -1.0 0.0
  * lon      (lon) float32 100.0 101.0 102.0 103.0 ... 157.0 158.0 159.0 160.0
  * phase    (phase) int64 0 1 2 3 4 5 6 7 8

In [90]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
import cartopy.mpl.ticker as cticker 


leftlon, rightlon, lowerlat, upperlat = (110,160,-23.5,-10)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()

title=[]
for i in range(1,9,1):
    title.append(f'Phase {i}')
   
fig = plt.figure(figsize=(12,8))

for i in range(4):
    for j in range(2):
        ax = fig.add_axes([0.1+j*0.475, 1.5-i*0.28, 0.4, 0.3],projection = ccrs.PlateCarree())
#         ax.set_title('({})'.format(title[i,j]),loc='left',fontsize=14)
        ax.set_extent(img_extent, crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE) 
        ax.set_xticks(np.arange(leftlon,rightlon+10,10), crs=ccrs.PlateCarree())
        ax.set_yticks(np.arange(lowerlat,upperlat+5,5), crs=ccrs.PlateCarree())
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)
        ax.title.set_text(title[i*2+j])
        c = ax.contourf(ratio['lon'],ratio['lat'],ratio_AU[i*2+j+1,:,:],
                        levels=np.arange(0,2,0.1),
                        extend='both' ,
                        cmap=cmaps.precip_diff_12lev,
                        transform=ccrs.PlateCarree()) 
        #绘制显著性打点。思路为将0-0.05范围内的区域用点的标记来填充，来表示显著性95%水平。
        cf2 =ax.contourf(ratio['lon'],ratio['lat'],sig_AU[i*2+j+1,:,:],[0.9,1],
                         zorder=1,hatches=['...', None],
                         colors="none", 
                         transform=ccrs.PlateCarree())
        
#colorbar, You would want to make sure the colorbar always sits in the same axes. 
# To this end, you may create a colorbar axes (cax) outside the loop
fig.colorbar(c, cax=fig.add_axes([0.26, 0.62, 0.5, 0.03 ]),
             orientation='horizontal',  
             label= 'Ratio', 
             ticks=np.arange(0.1,2.1,0.1),
             shrink=0.7)
st=fig.suptitle("MJO composites for daily-rainfall-prob under ElNinos (Annual-mean)",fontsize="x-large")
st.set_y(1.82)
fig.subplots_adjust(top=1.5)

plt.savefig("MJOcomp_daily_rain_prob_ANN_Elnino.pdf", format="pdf",bbox_inches = 'tight')

range(1981, 1984)

In [15]:
print(range(1981,1984,1))

range(1981, 1984)
